<a href="https://colab.research.google.com/github/Rodrigo2S/AtividadeColetadeDados/blob/main/AtividadeColetadeDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [10]:
PROJECT_ID = 'ds-project-353020'

In [11]:
QUERY = f'''WITH usuarios AS (
                SELECT
                    id as user_id,
                    operating_system,
                    salary,
                    credit_score,
                    date_birth,
                    DATE_DIFF(DATE(DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY)), DATE(date_birth), YEAR) as age
                FROM
                `ds-project-353020.app_data.users_profile`
), eventos AS (
                SELECT
                    user_id,
                    SUM(CASE WHEN event_name = 'app.home_view' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -14 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN 1 ELSE 0 END) AS qtd_home_view_last_7_days,
                    SUM(CASE WHEN event_name = 'app.home_view' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -22 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN 1 ELSE 0 END) AS qtd_home_view_last_15_days,
                    SUM(CASE WHEN event_name = 'app.search' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -14 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN 1 ELSE 0 END) AS qtd_searches_last_7_days,
                    SUM(CASE WHEN event_name = 'app.search' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -22 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN 1 ELSE 0 END) AS qtd_searches_last_15_days,
                    SUM(CASE WHEN event_name = 'app.search' AND section = 'restaurant' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -14 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN 1 ELSE 0 END) AS qtd_restaurant_searches_last_7_days,
                    SUM(CASE WHEN event_name = 'app.search' AND section = 'restaurant' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -22 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN 1 ELSE 0 END) AS qtd_restaurant_searches_last_15_days,
                    SUM(CASE WHEN event_name = 'app.search' AND section = 'supermarket' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -14 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN 1 ELSE 0 END) AS qtd_supermarket_searches_last_7_days,
                    SUM(CASE WHEN event_name = 'app.search' AND section = 'supermarket' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -22 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN 1 ELSE 0 END) AS qtd_supermarket_searches_last_15_days,
                    SUM(CASE WHEN event_name = 'app.added_to_cart' AND section = 'restaurant' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -14 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN total_value ELSE 0 END) AS sum_total_value_restaurant_add_cart_last_7_days,   
                    SUM(CASE WHEN event_name = 'app.added_to_cart' AND section = 'restaurant' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -22 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN total_value ELSE 0 END) AS sum_total_value_restaurant_add_cart_last_15_days,
                    SUM(CASE WHEN event_name = 'app.added_to_cart' AND section = 'supermarket' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -14 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN total_value ELSE 0 END) AS sum_total_value_supermarket_add_cart_last_7_days,   
                    SUM(CASE WHEN event_name = 'app.added_to_cart' AND section = 'supermarket' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -22 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN total_value ELSE 0 END) AS sum_total_value_supermarket_add_cart_last_15_days    
                FROM
                `ds-project-353020.app_data.events`
                WHERE DATE(date) <= DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) -- selecionando apenas eventos anteriores a data de corte, 7 dia antes da ultima data disponivel na base de dados
                GROUP BY user_id
), pedidos_anteriores AS (
                        SELECT 
                            user_id,
                            SUM(total_value) as sum_total_value_orders,
                            AVG(total_value) as avg_total_value_orders,
                            SUM(CASE WHEN section = 'restaurant' THEN total_value END ) AS sum_total_value_restaurant_orders,
                            SUM(CASE WHEN section = 'supermarket' THEN total_value END ) AS sum_total_value_supermarket_orders,  
                            AVG(CASE WHEN section = 'restaurant' THEN total_value END ) AS avg_total_value_restaurant_orders,
                            AVG(CASE WHEN section = 'supermarket' THEN total_value END ) AS avg_total_value_supermarket_orders,  
                            SUM(CASE WHEN DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -14 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN total_value ELSE 0 END) as sum_total_value_orders_last_7_days,
                            SUM(CASE WHEN DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -22 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN total_value ELSE 0 END) as sum_total_value_orders_last_15_days,
                            AVG(CASE WHEN DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -14 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN total_value ELSE 0 END) as avg_total_value_orders_last_7_days,
                            AVG(CASE WHEN DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -22 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN total_value ELSE 0 END) as avg_total_value_orders_last_15_days,
                            SUM(CASE WHEN section = 'restaurant' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -14 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN total_value ELSE 0 END) as sum_total_value_restaurant_orders_last_7_days,
                            SUM(CASE WHEN section = 'restaurant' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -22 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN total_value ELSE 0 END) as sum_total_value_restaurant_orders_last_15_days,
                            SUM(CASE WHEN section = 'supermarket' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -14 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN total_value ELSE 0 END) as sum_total_value_supermarket_orders_last_7_days,
                            SUM(CASE WHEN section = 'supermarket' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -22 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN total_value ELSE 0 END) as sum_total_value_supermarket_orders_last_15_days,
                            COUNT(*) as qtd_orders,
                            SUM(CASE WHEN DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -14 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN 1 ELSE 0 END) as qtd_orders_last_7_days,
                            SUM(CASE WHEN DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -22 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN 1 ELSE 0 END) as qtd_orders_last_15_days,
                            SUM(CASE WHEN section = 'restaurant' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -14 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN 1 ELSE 0 END) as qtd_restaurant_orders_last_7_days,
                            SUM(CASE WHEN section = 'restaurant' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -22 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN 1 ELSE 0 END) as qtd_restaurant_orders_last_15_days,
                            SUM(CASE WHEN section = 'supermarket' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -14 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN 1 ELSE 0 END) as qtd_supermarket_orders_last_7_days,
                            SUM(CASE WHEN section = 'supermarket' AND DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -22 DAY) AND DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) THEN 1 ELSE 0 END) as qtd_supermarket_orders_last_15_days,
                            SUM(CASE WHEN payment_method = 'credit_card' THEN 1 ELSE 0 END) / COALESCE(COUNT(*), 1) AS taxa_credit_card_payments
                        FROM
                            `ds-project-353020.app_data.orders`
                        WHERE DATE(date) <= DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) -- selecionando apenas pedidos anteriores a data de corte, 7 dia antes da ultima data disponivel na base de dados
                        GROUP BY user_id

), target AS (
            SELECT 
                user_id,
                MAX(CASE WHEN DATE(date) BETWEEN DATE_ADD(DATE "2021-05-31", INTERVAL -7 DAY) AND DATE('2021-05-31') THEN 1 ELSE 0 END) AS target
            FROM
            `ds-project-353020.app_data.orders`
            GROUP BY user_id
)
SELECT 
    usuarios.user_id,
    usuarios.operating_system,
    usuarios.salary,
    usuarios.credit_score,
    usuarios.age,
    eventos.qtd_home_view_last_7_days,
    eventos.qtd_home_view_last_15_days,
    eventos.qtd_searches_last_7_days,
    eventos.qtd_searches_last_15_days,
    eventos.qtd_restaurant_searches_last_7_days,
    eventos.qtd_restaurant_searches_last_15_days,
    eventos.qtd_supermarket_searches_last_7_days,
    eventos.qtd_supermarket_searches_last_15_days,
    eventos.sum_total_value_restaurant_add_cart_last_7_days,   
    eventos.sum_total_value_restaurant_add_cart_last_15_days,
    eventos.sum_total_value_supermarket_add_cart_last_7_days,   
    eventos.sum_total_value_supermarket_add_cart_last_15_days,    
    pedidos_anteriores.sum_total_value_orders,
    pedidos_anteriores.avg_total_value_orders,
    pedidos_anteriores.sum_total_value_restaurant_orders,
    pedidos_anteriores.sum_total_value_supermarket_orders,  
    pedidos_anteriores.avg_total_value_restaurant_orders,
    pedidos_anteriores.avg_total_value_supermarket_orders,  
    pedidos_anteriores.sum_total_value_orders_last_7_days,
    pedidos_anteriores.sum_total_value_orders_last_15_days,
    pedidos_anteriores.avg_total_value_orders_last_7_days,
    pedidos_anteriores.avg_total_value_orders_last_15_days,
    pedidos_anteriores.sum_total_value_restaurant_orders_last_7_days,
    pedidos_anteriores.sum_total_value_restaurant_orders_last_15_days,
    pedidos_anteriores.sum_total_value_supermarket_orders_last_7_days,
    pedidos_anteriores.sum_total_value_supermarket_orders_last_15_days,
    pedidos_anteriores.qtd_orders_last_7_days,
    pedidos_anteriores.qtd_orders_last_15_days,
    pedidos_anteriores.qtd_restaurant_orders_last_7_days,
    pedidos_anteriores.qtd_restaurant_orders_last_15_days,
    pedidos_anteriores.qtd_supermarket_orders_last_7_days,
    pedidos_anteriores.qtd_supermarket_orders_last_15_days,
    pedidos_anteriores.taxa_credit_card_payments,
    target.target
FROM 
usuarios
LEFT JOIN
eventos
ON usuarios.user_id = eventos.user_id
LEFT JOIN
pedidos_anteriores
ON usuarios.user_id = pedidos_anteriores.user_id
LEFT JOIN 
target
ON usuarios.user_id = target.user_id 
'''

In [12]:
df = pd.io.gbq.read_gbq(QUERY, project_id=PROJECT_ID)

**DATASET FINAL**

In [22]:
pd.options.display.max_columns = None
df.head()

,user_id,operating_system,salary,credit_score,age,qtd_home_view_last_7_days,qtd_home_view_last_15_days,qtd_searches_last_7_days,qtd_searches_last_15_days,qtd_restaurant_searches_last_7_days,qtd_restaurant_searches_last_15_days,qtd_supermarket_searches_last_7_days,qtd_supermarket_searches_last_15_days,sum_total_value_restaurant_add_cart_last_7_days,sum_total_value_restaurant_add_cart_last_15_days,sum_total_value_supermarket_add_cart_last_7_days,sum_total_value_supermarket_add_cart_last_15_days,sum_total_value_orders,avg_total_value_orders,sum_total_value_restaurant_orders,sum_total_value_supermarket_orders,avg_total_value_restaurant_orders,avg_total_value_supermarket_orders,sum_total_value_orders_last_7_days,sum_total_value_orders_last_15_days,avg_total_value_orders_last_7_days,avg_total_value_orders_last_15_days,sum_total_value_restaurant_orders_last_7_days,sum_total_value_restaurant_orders_last_15_days,sum_total_value_supermarket_orders_last_7_days,sum_total_value_supermarket_orders_last_15_days,qtd_orders_last_7_days,qtd_orders_last_15_days,qtd_restaurant_orders_last_7_days,qtd_restaurant_orders_last_15_days,qtd_supermarket_orders_last_7_days,qtd_supermarket_orders_last_15_days,taxa_credit_card_payments,target
0,8792,android,4682.76,781,47,4.0,12.0,8.0,15.0,2.0,3.0,1.0,4.0,84.01,366.20,11.00,208.17,2893.98,55.653462,533.99,284.00,16.181515,56.800000,1067.01,1971.89,20.519423,37.920962,84.01,366.20,11.00,208.17,15.0,38.0,10.0,26.0,2.0,4.0,0.769231,1.0
1,15536,ios,2675.78,551,47,5.0,12.0,23.0,45.0,7.0,18.0,9.0,15.0,212.00,406.34,901.00,969.58,1363.54,29.011489,504.90,750.64,22.950000,35.744762,752.96,1014.13,16.020426,21.577234,182.32,341.68,570.64,639.22,13.0,27.0,8.0,15.0,5.0,11.0,0.851064,1.0
2,16944,ios,3278.60,814,47,6.0,14.0,19.0,45.0,7.0,16.0,8.0,22.0,619.99,880.73,1530.01,1742.91,2754.75,45.159836,946.98,1799.77,31.566000,64.277500,1759.35,2192.71,28.841803,35.946066,505.18,733.32,1248.17,1451.39,17.0,39.0,8.0,19.0,8.0,17.0,0.786885,1.0
3,19636,ios,5209.68,606,47,0.0,0.0,12.0,16.0,10.0,14.0,0.0,0.0,986.00,2134.63,0.00,28.00,3008.99,158.367895,2980.99,28.00,165.610556,28.000000,986.00,2162.63,51.894737,113.822632,986.00,2134.63,0.00,28.00,9.0,16.0,9.0,15.0,0.0,1.0,0.736842,1.0
4,19294,ios,126.51,642,47,7.0,16.0,10.0,17.0,5.0,10.0,3.0,3.0,558.99,889.67,1240.00,2335.11,4149.73,125.749394,1130.17,3019.56,45.206800,377.445000,1798.99,2700.58,54.514848,81.835758,558.99,844.58,1240.00,1856.00,13.0,22.0,10.0,18.0,3.0,4.0,0.818182,1.0


QUANTIDADE DE TARGET 0

In [16]:
df['target'][0]

1.0

CORELAÇÃO

In [19]:
df.corr()

,user_id,salary,credit_score,age,qtd_home_view_last_7_days,qtd_home_view_last_15_days,qtd_searches_last_7_days,qtd_searches_last_15_days,qtd_restaurant_searches_last_7_days,qtd_restaurant_searches_last_15_days,qtd_supermarket_searches_last_7_days,qtd_supermarket_searches_last_15_days,sum_total_value_restaurant_add_cart_last_7_days,sum_total_value_restaurant_add_cart_last_15_days,sum_total_value_supermarket_add_cart_last_7_days,sum_total_value_supermarket_add_cart_last_15_days,sum_total_value_orders,avg_total_value_orders,sum_total_value_restaurant_orders,sum_total_value_supermarket_orders,avg_total_value_restaurant_orders,avg_total_value_supermarket_orders,sum_total_value_orders_last_7_days,sum_total_value_orders_last_15_days,avg_total_value_orders_last_7_days,avg_total_value_orders_last_15_days,sum_total_value_restaurant_orders_last_7_days,sum_total_value_restaurant_orders_last_15_days,sum_total_value_supermarket_orders_last_7_days,sum_total_value_supermarket_orders_last_15_days,qtd_orders_last_7_days,qtd_orders_last_15_days,qtd_restaurant_orders_last_7_days,qtd_restaurant_orders_last_15_days,qtd_supermarket_orders_last_7_days,qtd_supermarket_orders_last_15_days,taxa_credit_card_payments,target
user_id,1.000000,0.004988,0.002977,-0.000227,-0.072109,-0.098563,0.003860,0.001006,0.006217,-0.000012,-0.002710,-0.002015,0.004589,0.003960,0.013668,0.009075,0.006169,0.001366,0.005135,0.018410,0.004419,0.005436,0.017000,0.012583,0.010118,0.010296,0.006659,0.006748,0.015857,0.010225,0.015684,0.019244,0.011421,0.009673,0.003948,0.008449,0.019204,0.011655
salary,0.004988,1.000000,0.345381,0.001401,0.166205,0.161489,0.177866,0.181305,0.129032,0.130531,0.123741,0.126113,0.078739,0.080359,0.125679,0.135858,0.176548,0.067796,0.138258,0.129761,0.038535,0.066124,0.161226,0.173605,0.051038,0.063579,0.093384,0.096345,0.132741,0.146444,0.165458,0.172109,0.101487,0.099743,0.141549,0.150461,0.086249,0.069005
credit_score,0.002977,0.345381,1.000000,-0.002503,0.297088,0.296711,0.357336,0.365663,0.264205,0.270075,0.247988,0.254463,0.156521,0.167313,0.262203,0.280142,0.365757,0.134417,0.278829,0.288191,0.081238,0.114601,0.327758,0.351906,0.094754,0.112651,0.177610,0.189615,0.276319,0.300484,0.337785,0.356135,0.188152,0.195009,0.303616,0.321301,0.190706,0.130087
age,-0.000227,0.001401,-0.002503,1.000000,-0.012342,-0.013894,-0.001976,-0.001626,-0.006831,-0.007078,0.004735,0.008549,0.004386,0.002671,0.004376,0.001953,0.005517,0.009903,0.007184,0.008277,0.021948,0.003470,0.005428,0.003942,0.012066,0.009639,0.008320,0.007334,0.002055,0.000854,-0.000261,0.005346,-0.002111,0.002322,0.002450,0.003757,0.005687,0.003652
qtd_home_view_last_7_days,-0.072109,0.166205,0.297088,-0.012342,1.000000,0.919934,0.152474,0.157586,0.111237,0.116196,0.103992,0.107793,0.063952,0.063169,0.114184,0.132908,0.175186,0.057885,0.111316,0.162582,0.023910,0.066368,0.144377,0.163004,0.032562,0.044201,0.077527,0.077565,0.121450,0.145853,0.157812,0.168531,0.092110,0.094383,0.136882,0.146089,0.076213,0.053690
qtd_home_view_last_15_days,-0.098563,0.161489,0.296711,-0.013894,0.919934,1.000000,0.148707,0.154013,0.112075,0.117571,0.102152,0.105398,0.064788,0.065084,0.111192,0.129878,0.173977,0.058550,0.118316,0.157512,0.033520,0.064386,0.142337,0.161178,0.031786,0.042197,0.079027,0.079690,0.118285,0.142227,0.160490,0.168510,0.097358,0.097116,0.135179,0.144962,0.075693,0.054049
qtd_searches_last_7_days,0.003860,0.177866,0.357336,-0.001976,0.152474,0.148707,1.000000,0.924007,0.787114,0.720064,0.556005,0.521226,0.240461,0.249383,0.313111,0.324176,0.389413,0.092525,0.290254,0.257309,0.072996,0.068680,0.368669,0.380154,0.081067,0.080385,0.210391,0.221770,0.293197,0.303144,0.422703,0.445034,0.234587,0.253527,0.342946,0.352485,0.086178,0.139873
qtd_searches_last_15_days,0.001006,0.181305,0.365663,-0.001626,0.157586,0.154013,0.924007,1.000000,0.716497,0.783895,0.521981,0.556196,0.244378,0.255051,0.325863,0.334999,0.402675,0.097081,0.296555,0.274928,0.066270,0.078941,0.379103,0.391465,0.082627,0.08260

DESCRIÇÃO

In [21]:
df.describe()

,user_id,salary,credit_score,age,qtd_home_view_last_7_days,qtd_home_view_last_15_days,qtd_searches_last_7_days,qtd_searches_last_15_days,qtd_restaurant_searches_last_7_days,qtd_restaurant_searches_last_15_days,qtd_supermarket_searches_last_7_days,qtd_supermarket_searches_last_15_days,sum_total_value_restaurant_add_cart_last_7_days,sum_total_value_restaurant_add_cart_last_15_days,sum_total_value_supermarket_add_cart_last_7_days,sum_total_value_supermarket_add_cart_last_15_days,sum_total_value_orders,avg_total_value_orders,sum_total_value_restaurant_orders,sum_total_value_supermarket_orders,avg_total_value_restaurant_orders,avg_total_value_supermarket_orders,sum_total_value_orders_last_7_days,sum_total_value_orders_last_15_days,avg_total_value_orders_last_7_days,avg_total_value_orders_last_15_days,sum_total_value_restaurant_orders_last_7_days,sum_total_value_restaurant_orders_last_15_days,sum_total_value_supermarket_orders_last_7_days,sum_total_value_supermarket_orders_last_15_days,qtd_orders_last_7_days,qtd_orders_last_15_days,qtd_restaurant_orders_last_7_days,qtd_restaurant_orders_last_15_days,qtd_supermarket_orders_last_7_days,qtd_supermarket_orders_last_15_days,taxa_credit_card_payments,target
count,20000.000000,20000.000000,20000.000000,20000.000000,19668.000000,19668.000000,19668.000000,19668.000000,19668.000000,19668.000000,19668.000000,19668.000000,19668.000000,19668.000000,19668.000000,19668.000000,15397.000000,15397.000000,9872.000000,7581.000000,9872.000000,7581.00000,15397.000000,15397.000000,15397.000000,15397.000000,15397.000000,15397.000000,15397.000000,15397.000000,15397.000000,15397.000000,15397.000000,15397.000000,15397.000000,15397.000000,15397.000000,16442.000000
mean,10000.500000,3686.973268,614.450550,39.744350,2.132805,4.200986,6.044794,11.641296,3.300081,6.337655,1.382347,2.670175,160.129924,315.094253,197.517601,396.327015,1110.973720,163.533990,694.621692,1184.352370,135.491807,298.75765,391.927550,781.473574,56.988779,115.329663,158.057802,311.745101,205.282714,412.192612,2.759953,5.423394,1.454179,2.855556,0.829382,1.623238,0.721656,0.935774
std,5773.647028,3703.606235,71.920792,17.233585,2.711863,5.185904,4.909086,9.145420,3.540904,6.547629,2.279931,4.263726,354.124359,677.737958,497.366841,930.517560,1600.777059,213.980758,1046.781126,1558.222998,191.310783,338.69844,620.262257,1159.814349,104.536959,174.797768,341.877728,646.122017,491.642679,907.629871,2.994309,5.569123,2.288649,4.246839,1.565094,2.932817,0.263046,0.245162
min,1.000000,0.000000,424.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.400000,1.870000,0.400000,1.675000,0.40000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5000.750000,1066.070000,559.000000,25.000000,0.000000,0.000000,2.000000,5.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,185.000000,49.000000,135.000000,266.000000,39.439375,101.37800,33.640000,114.000000,6.665000,29.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.600000,1.000000
50%,10000.500000,2608.065000,601.000000,38.000000,1.000000,2.000000,5.000000,10.000000,2.000000,5.000000,0.000000,0.000000,0.000000,27.850000,0.000000,0.000000,533.200000,103.556667,335.675000,644.170000,78.498125,200.00000,157.500000,360.000000,27.857143,67.016190,0.000000,72.000000,0.000000,0.000000,2.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.750000,1.000000
75%,15000.250000,5143.165000,663.000000,52.000000,3.000000,7.000000,8.000000,16.000000,5.000000,9.000000,2.000000,4.000000,168.000000,344.105000,150.000000,375.000000,1354.000000,196.666667,796.947500,1485.000000,159.181786,377.00000,478.800000,957.200000,69.000000,137.246429,166.250000,340.500000,176.000000,429.000000,4.000000,7.000000,2.000000,4.000000,1.000000,2.000000,1.000000,1.000000
max,20000.000000,35